In [42]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor

In [34]:
suicide2005=pd.read_csv("2005.csv")
suicide2006=pd.read_csv("2006.csv")
suicide2007=pd.read_csv("2007.csv")
suicide2008=pd.read_csv("2008.csv")
suicide2009=pd.read_csv("2009.csv")
suicide2010=pd.read_csv("2010.csv")
suicide2011=pd.read_csv("2011.csv")

In [35]:
suicide2007.columns

Index(['Unnamed: 0', 'state', 'vet_pop', 'overall_pop_18', 'vet_pop_p',
       'vet_suicides', 'all_suicides', 'vet_suicides_p', 'vet_males',
       'vet_males_p', 'vet_females', 'vet_females_p', 'vet_15_24', 'vet_18_29',
       'vet_17_34', 'vet_25_34', 'vet_30_39', 'vet_25_44', 'vet_35_44',
       'vet_40_49', 'vet_35-54', 'vet_45_54', 'vet_50_59', 'vet_45_64',
       'vet_55_64', 'vet_60', 'vet_65', 'vet_rate', 'civ_rate'],
      dtype='object')

In [70]:
frames = [suicide2005, suicide2006, suicide2007, suicide2008, suicide2009, suicide2010, suicide2011]
bigdata = pd.concat(frames)
bigdata.head()
bigdata.to_csv("Cumulated_Data.csv", index=False)


In [71]:
Y = bigdata.vet_suicides
feature_names = ["state", "vet_pop", 
                 "vet_males", "vet_females", 
                 "vet_15_24", "vet_18_29", "vet_17_34", "vet_25_34", 
                 "vet_30_39", "vet_25_44", "vet_35_44", "vet_40_49", 
                 "vet_35-54", "vet_45_54", "vet_50_59", "vet_45_64", 
                 "vet_55_64", "vet_60", "vet_65"]

X = bigdata[feature_names]
print(X)

             state  vet_pop  vet_males  vet_females  vet_15_24  vet_18_29  \
0          Alabama   403950        NaN          NaN        NaN        NaN   
1           Alaska    74482       22.0          2.0        1.0        NaN   
2          Arizona   538880        NaN          NaN        NaN        NaN   
3         Arkansas   259304       79.0          2.0        NaN        NaN   
4       California  2193336      613.0         20.0       20.0        NaN   
5         Colorado   402091      183.0          8.0        7.0        NaN   
6      Connecticut   261294        NaN          NaN        NaN        NaN   
7         Delaware    79151       17.0          0.0        0.0        NaN   
8         Florida*  1717801        NaN          NaN        NaN        NaN   
9          Georgia   731466        NaN          NaN        3.0        NaN   
10          Hawaii   116793        NaN          NaN        NaN        NaN   
11           Idaho   132844       63.0          2.0        2.0        NaN   

In [72]:
bigdata.head()

,Unnamed: 0,state,vet_pop,overall_pop_18,vet_pop_p,vet_suicides,all_suicides,vet_suicides_p,vet_males,vet_males_p,...,vet_40_49,vet_35-54,vet_45_54,vet_50_59,vet_45_64,vet_55_64,vet_60,vet_65,vet_rate,civ_rate
0,0,Alabama,403950,3344721,0.120772,135,535,0.252336,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.419978,13.601875
1,1,Alaska,74482,446969,0.166638,24,131,0.183206,22.0,0.916667,...,NaN,NaN,9.0,NaN,NaN,5.0,NaN,4.0,32.222550,28.725835
2,2,Arizona,538880,4238996,0.127124,225,945,0.238095,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.753266,19.458849
3,3,Arkansas,259304,2023819,0.128126,81,400,0.202500,79.0,0.975309,...,NaN,NaN,NaN,NaN,NaN,16.0,NaN,25.0,31.237466,18.078622
4,4,California,2193336,25543447,0.085867,633,3206,0.197442,613.0,0.968404,...,NaN,NaN,NaN,NaN,220.0,NaN,NaN,317.0,28.860147,11.019220


In [75]:
bigdata = bigdata.fillna(lambda x: x.median())

In [76]:
bigdata.isnull().any()

Unnamed: 0        False
state             False
vet_pop           False
overall_pop_18    False
vet_pop_p         False
vet_suicides      False
all_suicides      False
vet_suicides_p    False
vet_males         False
vet_males_p       False
vet_females       False
vet_females_p     False
vet_15_24         False
vet_18_29         False
vet_17_34         False
vet_25_34         False
vet_30_39         False
vet_25_44         False
vet_35_44         False
vet_40_49         False
vet_35-54         False
vet_45_54         False
vet_50_59         False
vet_45_64         False
vet_55_64         False
vet_60            False
vet_65            False
vet_rate          False
civ_rate          False
dtype: bool

In [78]:
suicide_model = DecisionTreeRegressor(random_state=1)
suicide_model.fit(one_hot_encoded_training_predictors,Y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

,Unnamed: 0,state,vet_pop,overall_pop_18,vet_pop_p,vet_suicides,all_suicides,vet_suicides_p,vet_males,vet_males_p,...,vet_40_49,vet_35-54,vet_45_54,vet_50_59,vet_45_64,vet_55_64,vet_60,vet_65,vet_rate,civ_rate
0,0,Alabama,403950,3344721,0.120772,135,535,0.252336,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.419978,13.601875
1,1,Alaska,74482,446969,0.166638,24,131,0.183206,22.0,0.916667,...,NaN,NaN,9.0,NaN,NaN,5.0,NaN,4.0,32.222550,28.725835
2,2,Arizona,538880,4238996,0.127124,225,945,0.238095,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.753266,19.458849
3,3,Arkansas,259304,2023819,0.128126,81,400,0.202500,79.0,0.975309,...,NaN,NaN,NaN,NaN,NaN,16.0,NaN,25.0,31.237466,18.078622
4,4,California,2193336,25543447,0.085867,633,3206,0.197442,613.0,0.968404,...,NaN,NaN,NaN,NaN,220.0,NaN,NaN,317.0,28.860147,11.019220
